<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 3*

# 📝 Assignment
---

# Evaluation Metrics for Classification

### 1) Kaggle 챌린지를 이어서 진행합니다.

In [1]:
### 이곳에서 과제를 진행해 주세요 ### 

### 2) 본인이 개발한 모델에 대한 confusion matrix를 그려보세요. 
- (도전과제🔥) 각 평가지표에 대해 결과를 최대한 분석하고 무엇이 부족한지, 어떤 방향으로 성능을 높여야 하는지 논하세요.

In [2]:
### 이곳에서 과제를 진행해 주세요 ### 

### 3) 랜덤포레스트의 n_estimator를 10, 100, 1000으로 바꾸어 가며 predict_proba를 비교해 보세요

In [3]:
### 이곳에서 과제를 진행해 주세요 ### 

### 4) 두개 이상의 다른 분류 모델을 학습시켜 ROC curve, AUC 점수를 비교하고 설명합니다.

In [4]:
### 이곳에서 과제를 진행해 주세요 ### 

### 5) Kaggle 챌린지에 결과를 submission 합니다.

In [5]:
### 이곳에서 과제를 진행해 주세요 ### 